In [ ]:
# —————————————————————————————————————————————————————————————————————————
# 1. USER SETTINGS
# — Edit these to point at your file, variable, and (optional) index‐slices:

input_file       = "Enter_Input_File_Path.nc"
var_name         = "Variable Name"
output_file      = "Enter_Output_File_Path.png"     

# If your variable has extra dims (e.g. depth, time), pick them out by integer index:
# e.g. {"depth": 0, "time": 0}; set to {} if already 2D.
selection_idx    = {"depth": 0}

# Figure resolution & size
plot_title = " Plot Title"      #Add your plot title here
dpi              = 600
fig_width        = 14    # inches
fig_height       = 6     # inches

# —————————————————————————————————————————————————————————————————————————
# 2. COLORBAR OPTIONS
# — You can override the automatic vmin/vmax by setting these; set to None to use the data’s actual min/max.
cb_min           = None
cb_max           = None

# Number of ticks on the bar, label font size, and tri‐ended style:
cb_nticks        = 7
cb_label_size    = 8
cb_extend        = "both"    # draws little triangular arrows at each end

# thickness of the bar (fraction of parent axes height) and gap below map:
cb_fraction      = 0.04      # smaller → thinner bar
cb_pad           = 0.12      # gap between map and bar

# Color bar label
var_label = "Label"         # Add Your Color bar label

# How far below the bar to draw the stats annotation (fraction of axes)
annotation_fmt       = "Min = {vmin:.2f},  Max = {vmax:.2f},  Mean = {mean:.2f}"
annotation_fontsize  = 8
annotation_pad       = 0.121  # space below the bar before stats text

# Gap between the bar’s ticks and the variable‐name label
var_label_pad    = 5       # in points

# —————————————————————————————————————————————————————————————————————————
# 3. IMPORTS
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.ticker import FuncFormatter

# —————————————————————————————————————————————————————————————————————————
# 4. SCIENTIFIC FORMATTER
def scientific_formatter2(x, pos):
    """
    Turn a number into coefficient × 10^exponent notation.
    Handles zero and negative values.
    """
    if np.isnan(x):
        return " --- "
    if x == 0:
        return "0"
    sign = "-" if x < 0 else ""
    abs_x = abs(x)
    exponent = int(np.floor(np.log10(abs_x)))
    coefficient = abs_x / 10**exponent
    return r"${:.2f} \times 10^{{{}}}$".format(coefficient,exponent)


# —————————————————————————————————————————————————————————————————————————
# 5. LOAD & SLICE
ds = xr.open_dataset(input_file)
if var_name not in ds:
    raise KeyError(f"Variable '{var_name}' not found; available: {list(ds.data_vars)}")
data = ds[var_name]

# apply integer‐index slicing if requested
if selection_idx:
    data = data.isel(**selection_idx)

if data.ndim != 2:
    raise ValueError(f"After slicing, '{var_name}' has {data.ndim} dims; need 2D.")

# —————————————————————————————————————————————————————————————————————————
# 6. FIND LONGITUDE & LATITUDE
lon_dim = lat_dim = None
for d in data.dims:
    dn = d.lower()
    if dn.startswith("lon") or d == "XC":
        lon_dim = d
    elif dn.startswith("lat") or d == "YC":
        lat_dim = d

if not lon_dim or not lat_dim:
    raise ValueError(f"Could not detect lon/lat dims in {data.dims}")

lons = ds[lon_dim]
lats = ds[lat_dim]

# —————————————————————————————————————————————————————————————————————————
# 7. COMPUTE COLOR SCALE
raw_min = float(np.nanmin(data.values))
raw_max = float(np.nanmax(data.values))
vmin = raw_min if cb_min is None else cb_min
vmax = raw_max if cb_max is None else cb_max
mean = float(np.nanmean(data.values))

# —————————————————————————————————————————————————————————————————————————
# 8. DRAW THE MAP
fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

# Land / ocean / coast
ax.add_feature(cfeature.LAND.with_scale("110m"), facecolor="lightgray")
ax.add_feature(cfeature.OCEAN.with_scale("110m"), facecolor="white")
ax.coastlines(resolution="110m", linewidth=0.5)

# Simple gridlines (every 60° lon, 30° lat)
gl = ax.gridlines(
    draw_labels=True, linewidth=0.3, color="gray", linestyle="--", alpha=0.8
)
gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 30))
gl.ylocator = plt.FixedLocator(np.arange(-90,  91, 15))
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

# Plot data
im = ax.pcolormesh(
    lons, lats, data.values,
    transform=ccrs.PlateCarree(),
    cmap="RdBu_r",
    vmin=vmin, vmax=vmax,
)

# —————————————————————————————————————————————————————————————————————————
# 9. COLORBAR with extendable ends & custom size & length

left   = 0.315
bottom = 0.19
width  = 0.4  # from 0 to 1: wider → longer bar
height = 0.02  # thicker → taller bar

cax = fig.add_axes([left, bottom, width, height])
cbar = fig.colorbar(
    im, cax=cax,
    orientation='horizontal',
    extend='both',
    ticks=np.linspace(vmin, vmax, cb_nticks)
)

cbar.ax.xaxis.set_major_formatter(FuncFormatter(scientific_formatter2))
cbar.ax.tick_params(labelsize=cb_label_size)
cbar.set_label(var_label, size=cb_label_size, labelpad=var_label_pad)


# —————————————————————————————————————————————————————————————————————————
# 10. ANNOTATION of stats
ax.set_title(plot_title, fontsize=10, pad=10, fontweight = 'bold')
min_str  = scientific_formatter2(vmin, None)
max_str  = scientific_formatter2(vmax, None)
mean_str = scientific_formatter2(mean, None)
annotation_text = f"Min = {min_str},  Max = {max_str},  Mean = {mean_str}"

ax.annotate(
    annotation_text,
    xy=(0.5, -annotation_pad),
    xycoords="axes fraction",
    ha="center",
    fontsize=annotation_fontsize
)

# make room at bottom for colorbar + annotation
plt.subplots_adjust(bottom=cb_pad + annotation_pad + 0.05)

# —————————————————————————————————————————————————————————————————————————
# 11. SAVE & SHOW
# plt.savefig(output_file, bbox_inches="tight", dpi=dpi)
plt.show()
plt.close(fig)

print(f"✔ Saved: {output_file}")
